Название: recognize_speech.ipynb  

Автор: hwpy  
Дата: 2022 Март  
Описание: Пакет функций для транскрипции wav  
Примечания:  
* Аудио для демонстрации взяты с открытого источника Russian Open Speech To Text (STT/ASR) Dataset  
        https://www.kaggle.com/datasets/tapakah68/audio-dataset?resource=download&utm_source=pocket_mylist  
* Перед началом работы необходимо клонировать git clone https://github.com/alphacep/vosk-api  
        или загрузить полную русскоязычную модель https://alphacephei.com/vosk/models  
        и распаковать в каталог model

# Функции для взаимодействия с моделями

**Импортируем модули и библиотеки**

In [1]:
from IPython.display import display
# для аннотаций
from typing import List
# для работы с данными
import pandas as pd
# для работы с файлами
import os
import wave
# для транскрипции
from vosk import Model, KaldiRecognizer
import speech_recognition as speech_r
import json
# остальные
from tqdm import tqdm
# для работы с архивами
from packages.helper import recursive_unpack, single_unpack

**Определение функций и переменных**

In [2]:
# путь к каталогу audio
CUR_DIR = os.getcwd()
PATH_TO_AUDIO = os.path.join(CUR_DIR, "audio")
# частота дискретизации
SAMPLE_RATE = 16000


def create_list_of_wavs(path: str = PATH_TO_AUDIO) -> List[str]:
    """Создание списка путей до wav файлов, лежащих в любом уровне внутри PATH_TO_AUDIO

    Аргументы:
        - path (str) - путь до каталога wav (по умолчанию: {PATH_TO_AUDIO})

    Возвращает:
        wav_files (List[str]) - Список wav файлов, с которыми будем работать
    """
    # будущий список wav
    wav_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith(".wav"):
                wav_files.append(os.path.join(root, file))
    return wav_files


def get_files_sample_rate(path_to_audio: str = PATH_TO_AUDIO) -> pd.DataFrame:
    """Получить список частот дескритезации аудиофайлов в катологе path_to_audio

    Аргументы:
        - path_to_audio (str) - каталог wav файлов (по умолчанию: {PATH_TO_AUDIO})

    Возвращает:
        - df_sample_rates (pd.DataFrame) - фрейм из файлов и их частотами дискретизации
    """
    df_sample_rates = pd.DataFrame(columns=['audiofile_name', 'sample_rate'])
    for root, _, files in os.walk(path_to_audio):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".wav"):
                with wave.open(file_path, "rb") as wave_file:
                    sample_rate = wave_file.getframerate()
                    # print(file, sample_rate)
                    df_sample_rates =\
                        df_sample_rates.append(
                            {
                                'audiofile_name': file,
                                'sample_rate': sample_rate
                            },
                            ignore_index=True
                        )
    return df_sample_rates


def recognize_with_speech_recognition(path_to_audio: str = PATH_TO_AUDIO) -> None:
    """Транскрипция с помощью модуля speech_recognition

    Аргументы:
        - path_to_audio (str) (по умолчанию: {PATH_TO_AUDIO})

    Возвращает:
        None
    """
    # создаем экземпляр класса Recognizer
    recognizer = speech_r.Recognizer()

    for root, _, files in os.walk(path_to_audio):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".wav"):
                # Create audio file instance from the original file
                call_record = speech_r.AudioFile(file_path)
                type(call_record)
                # Create audio data
                with call_record as source:  # pylint: disable=unused-variable
                    recognizer.adjust_for_ambient_noise(call_record)
                    audiodata = recognizer.record(call_record)
                # type(audiodata)

                try:
                    print("Google определил текст: " + recognizer.recognize_google(audiodata, language='ru'))
                except speech_r.UnknownValueError as e:
                    print("Ошибка распознавания" + str(e))

                # не удалось запустить на macos
                # try:
                #     print("Sphinx определил текст: " + recognizer.recognize_sphinx(audiodata, language='ru'))
                # except speech_r.UnknownValueError as e:
                #     print("Ошибка распознавания" + str(e))


def recognize_with_vosk(file: str, sample_rate: int = SAMPLE_RATE) -> pd.DataFrame:
    """Транскрипция файлов из file_list и запись имени файла и его текста в pd.DataFrame

        - file (str) - путь к файлу

    Возвращает:
        - transcribed (pd.DataFrame) - фрейм с именами файлов и их транскрипцией
    """
    # проверка на существование каталога модели
    if not os.path.exists('model'):
        print("Необходимо загрузить модель с: https://alphacephei.com/vosk/models\
            и распаковать как 'model' в текущую папку.")
    else:
        # экземпляр модели
        model = Model('model')
        # экземпляр рекогнайзера
        rec = KaldiRecognizer(model, sample_rate)
        # dataframe для хранения транскрипции текста
        transcribed = pd.DataFrame(columns=['audiofile_name', 'raw_text'])
        print(file)
        last_n = False
        recognized_data = ''
        wave_audio_file = wave.open(file, "rb")
        while True:
            data = wave_audio_file.readframes(wave_audio_file.getnframes())
            if len(data) == 0:
                break

            if rec.AcceptWaveform(data):
                res = json.loads(rec.Result())
            # recognized_data = json.loads(rec.Result())["text"]
                if res['text'] != '':
                    recognized_data += f" {res['text']}"
                    last_n = False
                elif not last_n:
                    recognized_data += '\n'
                    last_n = True

        res = json.loads(rec.FinalResult())
        recognized_data += f" {res['text']}"

        print(recognized_data)
        _, audiofile_name = os.path.split(file)

        transcribed =\
            transcribed.append(
                {
                    'audiofile_name': audiofile_name,
                    'raw_text': recognized_data
                },
                ignore_index=True
            )
    return transcribed


# если датасет лежит во вложенных архивах - разархивируем
# recursive_unpack(path=PATH_TO_AUDIO)
# если датасет лежит в архиве без вложенных архивов - разархивируем
# single_unpack(path=PATH_TO_AUDIO)

# Транскрипция

**Применим объявленные функции и посмотрим на результат**

In [3]:
# смотрим sample rates
df_samplerates = get_files_sample_rate(path_to_audio=PATH_TO_AUDIO)
print(df_samplerates)
# транскрипция с помощью speech_recognition
recognize_with_speech_recognition(PATH_TO_AUDIO)
# создаем список wav файлов в каталоге audio
list_of_wavs = create_list_of_wavs(PATH_TO_AUDIO)
# транскрипция списка с помощью vosk
df_transcribed = pd.DataFrame(columns=['audiofile_name', 'raw_text'])
for file in tqdm(list_of_wavs):
    df_transcribed = pd.concat([df_transcribed, recognize_with_vosk(file, SAMPLE_RATE)])
df_transcribed.reset_index(inplace=True, drop=True)
display(df_transcribed)
# выгружаем результат
df_transcribed.to_excel(r"excel/transcribed.xlsx")

C:\Users\Akulov.as\AppData\Local\Temp\ipykernel_11124\385624108.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sample_rates.append(
C:\Users\Akulov.as\AppData\Local\Temp\ipykernel_11124\385624108.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sample_rates.append(


                             audiofile_name sample_rate
0  0003d952-f170-4af0-946f-35db983ac54a.wav       16000
1  0007b919-85dc-43f6-a1c8-80fdff3d0048.wav       16000
Google определил текст: легендарный католический Святой ставший основателем нищенствующие ордена францисканцев на первый взгляд является классическим шизофреником
Google определил текст: это антибактериальное действие является постоянным и устойчивым многократной стирки


  0%|          | 0/2 [00:00<?, ?it/s]

c:\GitHub\doc_classifier\audio\0003d952-f170-4af0-946f-35db983ac54a.wav


C:\Users\Akulov.as\AppData\Local\Temp\ipykernel_11124\385624108.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transcribed.append(
 50%|█████     | 1/2 [01:48<01:48, 108.56s/it]

 легендарный католический святой ставший основателем нищенствующего ордена францисканцев на первый взгляд является классическим шизофреником
c:\GitHub\doc_classifier\audio\0007b919-85dc-43f6-a1c8-80fdff3d0048.wav


C:\Users\Akulov.as\AppData\Local\Temp\ipykernel_11124\385624108.py:136: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transcribed.append(
100%|██████████| 2/2 [04:23<00:00, 131.84s/it]

 это антибактериальное действие является постоянным и устойчивым к многократной стирке


,audiofile_name,raw_text
0,0003d952-f170-4af0-946f-35db983ac54a.wav,легендарный католический святой ставший основ...
1,0007b919-85dc-43f6-a1c8-80fdff3d0048.wav,это антибактериальное действие является посто...


# Выводы:  
**1. Google**  

Преимущества:  
- справляется относительно хорошо, но бывает ошибается в падежах,  
- занимает мало места на диске,  
- просто использовать.  

Недостатки:  
- отправка данных на сервер для обработки, не подходит по критериям,
- платное использование сверх n взаимодействий в день, не подходит по критериям.

**2. Sphinx**  

Преимущества:  
- занимает мало места на диске,  
- просто использовать.  

Недостатки:  
- справляется относительно плохо, транскрибирует неправильно значительную часть аудио.  

**3. VOSK**  

Преимущества: 
- справляется относительно хорошо,  
- есть возможность адаптации модели под свои потребности,  
- есть возможность загрузить "маленькую" и "большую" модель  
    в зависимости от устройства использования и желаемой точности  

Недостатки:
- возможно потребуется некоторая предобработка кодеков аудио  

В этом примере предпочтительно было использовать VOSK API.